# №10. Борьба с переобучением. Dropout

1. https://habr.com/ru/companies/mvideo/articles/782360
2. https://habr.com/ru/companies/wunderfund/articles/330814
3. ...

## 1. Краткая теория

*Схема работы метода **Dropout**:*

<img src="data/my_images/dropout_schema.png" width=600>

Главная идея Dropout — вместо обучения одной DNN обучить ансамбль нескольких DNN, а затем усреднить полученные результаты.

Сети для обучения получаются с помощью исключения из сети (dropping out) нейронов с вероятностью $p$, таким образом, вероятность того, что нейрон останется в сети, составляет $q=1-p$. “Исключение” нейрона означает, что при любых входных данных или параметрах он возвращает 0.

Исключенные нейроны не вносят свой вклад в процесс обучения ни на одном из этапов алгоритма обратного распространения ошибки (backpropagation); поэтому исключение хотя бы одного из нейронов равносильно обучению новой нейронной сети.

[Цитируя авторов](https://arxiv.org/abs/1207.0580),

> В стандартной нейронной сети производная, полученная каждым параметром, сообщает ему, как он должен измениться, чтобы, учитывая деятельность остальных блоков, минимизировать функцию конечных потерь. Поэтому блоки могут меняться, исправляя при этом ошибки других блоков. Это может привести к чрезмерной совместной адаптации (co-adaptation), что, в свою очередь, приводит к переобучению, поскольку эти совместные адаптации невозможно обобщить на данные, не участвовавшие в обучении. Мы выдвигаем гипотезу, что Dropout предотвращает совместную адаптацию для каждого скрытого блока, делая присутствие других скрытых блоков ненадежным. Поэтому скрытый блок не может полагаться на другие блоки в исправлении собственных ошибок.

## 2. Использование в `tocrh`

In [1]:
import torch
import torch.nn as nn

In [45]:
# Создание слоя dropout
from sympy import N

p = 0.5
dropout = nn.Dropout(p=p)  # вероятность отключения p

# Применение к тензору
x = torch.tensor([[1.0, 2.0, 3.0, 4.0],
                  [5.0, 6.0, 7.0, 8.0]])
print("x: ", x, x.mean())
output = dropout(x)
print("out: ", output, output.mean())

n=1000
mean_list = []
for i in range(n):
    output = dropout(x)
    mean_list.append(output.mean())
    
print("mean: ", sum(mean_list) / n, x.mean())

x:  tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]]) tensor(4.5000)
out:  tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]]) tensor(0.)
mean:  tensor(4.5037) tensor(4.5000)


**Примечание**: ***dropout*** учсаствует в процессе обучения (`model.train()`), однако при использовании модели (или ее тестировании, валидации), необходимо перевести модель в режим работы (`model.eval()`), так как нам необходимы все нейроны для работы сети.

In [ ]:
class myRegModel(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        hidden_size = 5
        self.layer_1 = nn.Linear(input, hidden_size)
        self.dp = nn.Dropout(p=0.5)
        self.layer_2 = nn.Linear(hidden_size, output)
        self.act_func = nn.ReLU()
        
    def forward(self, x):
        print(f"input: ", x)
        x = self.layer_1(x)
        print(f"after layer_1: ", x)
        x = self.dp(x)
        print(f"after_dropout: ", x)
        x = self.act_func(x)
        print(f"after ReLU: ", x)
        out = self.layer_2(x)
        print(f"after layer_2 (out): ", out)
        return out

In [55]:
x = torch.rand([1, 10])
reg_model = myRegModel(10, 2)

In [63]:
reg_model.eval()
reg_model(x)

input:  tensor([[0.8625, 0.3690, 0.2415, 0.4812, 0.3050, 0.6386, 0.3207, 0.4484, 0.8921,
         0.1570]])
after layer_1:  tensor([[ 0.0272, -0.5179,  0.0742, -0.4964, -0.5511]],
       grad_fn=<AddmmBackward0>)
after_dropout:  tensor([[ 0.0272, -0.5179,  0.0742, -0.4964, -0.5511]],
       grad_fn=<AddmmBackward0>)
after ReLU:  tensor([[0.0272, 0.0000, 0.0742, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
after layer_2 (out):  tensor([[-0.1911,  0.3191]], grad_fn=<AddmmBackward0>)


tensor([[-0.1911,  0.3191]], grad_fn=<AddmmBackward0>)

In [65]:
reg_model.train()
reg_model(x)

input:  tensor([[0.8625, 0.3690, 0.2415, 0.4812, 0.3050, 0.6386, 0.3207, 0.4484, 0.8921,
         0.1570]])
after layer_1:  tensor([[ 0.0272, -0.5179,  0.0742, -0.4964, -0.5511]],
       grad_fn=<AddmmBackward0>)
after_dropout:  tensor([[ 0.0000, -0.0000,  0.0000, -0.9927, -1.1022]], grad_fn=<MulBackward0>)
after ReLU:  tensor([[0., -0., 0., 0., 0.]], grad_fn=<ReluBackward0>)
after layer_2 (out):  tensor([[-0.2121,  0.2937]], grad_fn=<AddmmBackward0>)


tensor([[-0.2121,  0.2937]], grad_fn=<AddmmBackward0>)

## 3. Итоги

Dropout существует в двух модификациях: прямой (используется нечасто) и обратный.

Dropout на отдельном нейроне может быть представлен как случайная величина с распределением Бернулли.

Dropout на множестве нейронов может быть представлен как случайная величина с биномиальным распределением.

Несмотря на то, что вероятность того, что из сети будет выключено ровно np нейронов, np — среднее количество нейронов, отключенных в слое из n нейронов.

Обратный Dropout увеличивает скорость обучения.

Обратный Dropout следует использовать совместно с другими методами нормализации, ограничивающими значения параметров, чтобы упростить процесс выбора скорости обучения.

Dropout помогает предотвратить проблему обучения в глубоких нейронных сетях.